In [1]:
import pandas as pd
df = pd.read_csv('clickbait_dataset.csv')
df.head()


,headline,clickbait
0,This Is What $1 USD Gets You In Food All Aroun...,1
1,Make These Easy Chicken Fajita Quesadillas At ...,1
2,"The Hardest ""Walking Dead"" Video Game Quiz You...",1
3,34 Online Shops Based In The Southeast You Sho...,1
4,US and France to work together for new Iran sa...,0


In [2]:
df = df.dropna()


In [9]:
import ssl
import nltk

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('punkt')  # Asegúrate de que 'punkt' es el paquete correcto


[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>


False

In [3]:
import numpy as np
embeddings ={}
with open('/Users/jorgejimenez/Documents/UP/NLP/simcos/glove.42B.300d.txt', encoding='utf-8') as f:
  for line in f:
    values = line.split()
    word = values[0]
    vectors = np.asarray(values[1:])
    embeddings[word] = vectors

In [4]:
import contractions               #Won't -> will not
import re                         #Regex
import nltk                       #Tokenizar texto
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from tqdm import tqdm

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')

stopwords_en = stopwords.words('english')
lemmatizer = WordNetLemmatizer()
tqdm.pandas()

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>
[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>
[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>
[nltk_data] Error loading punkt_tab: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>


In [5]:
def vectorize(texto):
  vector_size = 300 #Dependiento del archivo de glove -> glove.6B.300d.txt

  #Todo a minusculas
  texto = texto.lower()

  #Contracciones
  texto = contractions.fix(texto)

  #Regex para quitar todo lo que no sea letra
  texto = re.sub(r'[^a-z\s]','', texto)

  #Quitar stop words
  texto = word_tokenize(texto)

  #Eliminar stopwords
  texto = [token for token in texto if token not in stopwords_en]

  #Lemmatizar
  texto = [lemmatizer.lemmatize(word) for word in texto]

  #Generar vector (sumar linealmente)
  vector = np.zeros(vector_size)

  for word in texto:
    if word in embeddings:
      vector += embeddings[word].astype(float)

  vector = vector.reshape(1, -1)[0] #Una fila, columans necesarias

  return vector

In [11]:
from tqdm import tqdm
nltk.download('punk_tab')
tqdm.pandas()
df['vector'] = df['headline'].progress_apply(vectorize)


[nltk_data] Error loading punk_tab: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>
100%|██████████| 32000/32000 [00:20<00:00, 1573.67it/s]


In [12]:
df.head()

,headline,clickbait,vector
0,This Is What $1 USD Gets You In Food All Aroun...,1,"[0.3411369999999999, 0.016487000000000016, 0.2..."
1,Make These Easy Chicken Fajita Quesadillas At ...,1,"[-0.40621599999999997, -1.1668068399999998, 1...."
2,"The Hardest ""Walking Dead"" Video Game Quiz You...",1,"[-2.3220799999999997, 2.40599, 1.121831, -0.07..."
3,34 Online Shops Based In The Southeast You Sho...,1,"[1.675697, 0.37274699999999994, 0.892619, -1.4..."
4,US and France to work together for new Iran sa...,0,"[0.31207199999999996, 2.1173759999999997, -0.1..."


In [13]:
from sklearn.model_selection import train_test_split
X = df['vector']
X = np.concatenate(X, axis = 0).reshape(-1, 300)
y = df['clickbait']
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.7,
                                                    random_state = 101)
